<a href="https://colab.research.google.com/github/UtG1209/Supply-Chain-Optimization/blob/main/Route%20optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np # linear algebra
import pandas as pd

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [37]:
!pip install ortools

In [38]:
!pip install folium

In [39]:
!pip install plotly

In [40]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Locations and distances (hypothetical data)
    data['locations'] = [
        (10, 0),     # Warehouse 1 (Depot)
        (1, 111),     # Warehouse 2 (Depot)
        (70, 21),     # Warehouse 3 (Depot)
        (3, 13),     # Location 1
        (4, 4),     # Location 2
        (52, 5),     # Location 3
        (69, 6),     # Location 4
        (7, 72),     # Location 5
        (8, 8),     # Location 6
        (91, 9),     # Location 7
        (10, 10),   # Location 8
        (11, 11),   # Location 9
        (12, 112),   # Location 10
        (132, 13),   # Location 11
        (14, 14),   # Location 12
        (15, 135)    # Location 13
    ]
    data['num_locations'] = len(data['locations'])
    data['num_vehicles'] = 3
    data['starts'] = [0, 1, 2]  # Start depot indices
    data['ends'] = [0, 1, 2]    # End depot indices

    return data

def compute_euclidean_distance_matrix(locations):
    """Creates callback to return distance between locations."""
    num_locations = len(locations)
    distance_matrix = {}

    for from_node in range(num_locations):
        distance_matrix[from_node] = {}
        for to_node in range(num_locations):
            if from_node == to_node:
                distance_matrix[from_node][to_node] = 0
            else:
                distance_matrix[from_node][to_node] = (
                    abs(locations[to_node][0] - locations[from_node][0]) +
                    abs(locations[to_node][1] - locations[from_node][1]))
    return distance_matrix

def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        data['num_locations'], data['num_vehicles'], data['starts'], data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    distance_matrix = compute_euclidean_distance_matrix(data['locations'])

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        total_distance = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
            route_distance = 0
            while not routing.IsEnd(index):
                plan_output += ' {} -> '.format(manager.IndexToNode(index))
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            plan_output += '{}\n'.format(manager.IndexToNode(index))
            plan_output += 'Distance of the route: {}m\n'.format(route_distance)
            print(plan_output)
            total_distance += route_distance
        print('Total Distance of all routes: {}m'.format(total_distance))
    else:
        print('No solution found!')

if __name__ == '__main__':
    main()


Route for vehicle 0:
 0 ->  4 ->  3 ->  14 ->  11 ->  10 ->  8 -> 0
Distance of the route: 54m

Route for vehicle 1:
 1 ->  7 ->  12 ->  15 -> 1
Distance of the route: 154m

Route for vehicle 2:
 2 ->  13 ->  9 ->  6 ->  5 -> 2
Distance of the route: 192m

Total Distance of all routes: 400m


In [41]:
import plotly.graph_objects as go
import numpy as np

def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        data['num_locations'], data['num_vehicles'], data['starts'], data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    distance_matrix = compute_euclidean_distance_matrix(data['locations'])

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return distance_matrix[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Extract and plot routes using Plotly.
    if solution:
        fig = go.Figure()

        # Plot all locations
        for i, location in enumerate(data['locations']):
            x, y = location
            fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', name=f'Location {i}',
                                     marker=dict(size=10, color='blue')))

        # Plot depots (warehouses)
        for depot_index in data['starts']:
            x, y = data['locations'][depot_index]
            fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', name=f'Depot {depot_index}',
                                     marker=dict(size=12, color='red')))

        # Plot routes for each vehicle
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            route_x = []
            route_y = []
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                x, y = data['locations'][node_index]
                route_x.append(x)
                route_y.append(y)
                index = solution.Value(routing.NextVar(index))
            # Add the route back to the depot
            node_index = manager.IndexToNode(index)
            x, y = data['locations'][node_index]
            route_x.append(x)
            route_y.append(y)
            route_x.append(data['locations'][manager.IndexToNode(routing.Start(vehicle_id))][0])
            route_y.append(data['locations'][manager.IndexToNode(routing.Start(vehicle_id))][1])
            fig.add_trace(go.Scatter(x=route_x, y=route_y, mode='lines+markers', name=f'Vehicle {vehicle_id+1} Route',
                                     line=dict(width=2)))

        # Customize layout
        fig.update_layout(title='Vehicle Routing Problem Solution',
                          xaxis_title='X Coordinate', yaxis_title='Y Coordinate',
                          showlegend=True)

        # Show plot
        fig.show()

        # Print total distance
        total_distance = sum([routing.GetArcCostForVehicle(
            manager.IndexToNode(index), solution.Value(routing.NextVar(index)), vehicle_id)
            for vehicle_id in range(data['num_vehicles'])
            for index in range(routing.Size()) if routing.IsVehicleUsed(solution, vehicle_id)])
        print(f'Total Distance of all routes: {total_distance}m')

    else:
        print('No solution found!')

if __name__ == '__main__':
    main()


Total Distance of all routes: 1200m
